#### Подготовка данных

Изначальный датасет довольно избыточный, поэтому для проведения задания мы пользуемся этим ноутбуком, чтобы взять только нужную часть

Скачать весь датасет (~5 Гб) можно по этой ссылке: https://drive.google.com/file/d/1CUZnBtYwifVXS21yVg62T-vrPVayso5H/view
Файл с отсутствующими скелетами: https://github.com/shahroudy/NTURGB-D/blob/master/Matlab/NTU_RGBD_samples_with_missing_skeletons.txt

In [1]:
data_path = r'F:\nturgbd_skeletons_s001_to_s017\nturgb+d_skeletons'
#### список отсутсвующих элементов так же будет доступен 
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

In [2]:
training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = [8, 10, 22, 23, 27, 21, 25, 31,] #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
training_cameras = [1, 2, 3] 

max_body_true = 1
max_body_kinect = 1

num_joint = 25
max_frame = 300 # Длина отрезка которую мы вычленяем из большого датасета


In [3]:
import os
import pandas as pd

Есть функция, которая позволяет нам считать данные для каждого класса. 

In [4]:
def read_data(data_path, broken_files_path):
    labels = []
    files = []
    action_classes = {}
    counter = 0
    files_counter = {}
              
    with open(broken_files_path, 'r') as f:
        broken_files = f.read().split("\n")

    raw_files = os.listdir(data_path)
    num_frames = 0

    for filename in raw_files:
        if filename not in broken_files:
            action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
            subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
            camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
            if action_class in training_classes and camera_id in training_cameras:  #and subject_id in training_subjects:
                if action_class in action_classes:
                    if files_counter[action_class] < 120:
                        files.append([filename,action_classes[action_class]])
                        files_counter[action_class] = files_counter[action_class] + 1
                else:
                    action_classes.update({action_class : counter})
                    files_counter.update({action_class : 1})
                    counter+=1
                    files.append([filename,action_classes[action_class]])
#                     labels.append([action_class])
    print("action classes: ", action_classes)
    print("action files: ", files_counter)
    
    return files, action_classes

In [5]:
def get_nonzero_std(s): 
    index = s.sum(-1).sum(-1) != 0  
    s = s[index]
    if len(s) != 0:
        s = s[:, :, 0].std() + s[:, :, 1].std() + s[:, :, 2].std()  
    else:
        s = 0
    return s

# Функция фильтр для того, что бы найти только координаты x,y,z(т.к. в датасете
# хранится на порядок больше информации, нежели только координаты x,y,z)
# (Остальные данные нам и не нужны, т.к. у нас нет ик-камер)
def read_skeleton_filter(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []

            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

def read_xyz(file, max_body=1, num_joint=25):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
    for n, f in enumerate(seq_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < num_joint:
                    data[m, n, j, :] = [v['x'], v['y'], v['z']]

                else:
                    pass

    return data

### В этой функции меняем количество фреймов подаваемых на вход модели

В задании нужно пронаблюдать зависимость качества обучения модели от количества кадров, которые мы подаем в модель. Т.е. то, чколько кадров у нас отведено на один блок. В ноутбуке мы обучали модель на 45 кадрах. Вам предлагается посмотреть в меньшую или большую сторону.

По хорошему функцию ниже нужно прописать внутри определения датасета.

In [6]:
def create_coords_blocks(test_file, chonk_len = 45):   
    frame_counter = 0
    new_labels = []
    new_frames = []
    blocks = []
    
    test_frames = read_xyz(data_path +'/'+ test_file[0])[0]
    label = test_file[1]
    slice_len = chonk_len * int(len(test_frames)/chonk_len)


    for index in range(len(test_frames[:slice_len])):
        frame_counter += 1
        new_frames.append(test_frames[index].flatten())
        if frame_counter == chonk_len:
            frame_counter = 0
            blocks.append(np.array(new_frames))
            new_labels = new_labels + [label]
            new_frames = []
       
            
    return blocks, new_labels

In [7]:
##### список файлов с лейблами на каждый файл 
working_files_with_labels, action_classes = read_data(data_path, broken_files_path)

action classes:  {8: 0, 10: 1, 21: 2, 22: 3, 23: 4, 25: 5, 27: 6, 31: 7}
action files:  {8: 120, 10: 120, 21: 120, 22: 120, 23: 120, 25: 120, 27: 120, 31: 120}


In [8]:
LABELS = {v: k for k, v in action_classes.items()}

### Здесь выносится сгенерированный список лейблов с номером класса. Эту переменную можно перенести ручками, а можно автоматизировать. Тут на ваше усмотрение, автору немного лень :) 

In [9]:
print(LABELS)

{0: 8, 1: 10, 2: 21, 3: 22, 4: 23, 5: 25, 6: 27, 7: 31}


In [10]:
len(working_files_with_labels)

960

In [11]:
import numpy as np

In [12]:
data = []
labels = []
##########################################################################
numbers = {x: 0 for x in range(len(action_classes))} #####
##################################################################
for file in working_files_with_labels:
    frames_blocks, label = create_coords_blocks(file)
    if label != [] and numbers[label[0]] <= 150:
        numbers[label[0]] = numbers[label[0]] + len(label)
        data = data + frames_blocks
        labels = labels + label
data_np = np.asarray(data)
labels_np = np.asarray(labels)

data_sq = data_np.reshape(len(data_np), -1)
test_data = pd.DataFrame(data_sq)
test_labels = pd.DataFrame(labels_np)
test_data['labels'] = test_labels

### Ваш итоговый файл для загрузки на колаб

In [13]:
test_data.to_csv("skeletons_classes_1_30.csv", index = False)

Обучить уже существующую модель (предварительно проанализировав какие параметры модели нужно изменить)

In [16]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [17]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [19]:
from torch.utils.data import DataLoader, Dataset

In [20]:
class Test_Dataset():
    def __init__(self):
        self.samples = list(range(0, 100)) ##генерируем случайные данные
        # Все подготовления данных происходят в инициализации:
        # - разделение данных на train/test/ validate
        # - аугментация (трансформация) данных 
        # - чтние аннотаций
        # - считывание по индексу
         # - Применяет выбранные трансформации к данным

    def __len__(self):
        # # Возвращает данные и лейблы
       
        # - отправление данных на гпу
        # - лучше возвращать данные как тензор или словарь
         # - Применяет выбранные трансформации к данным
        return len(self.samples) #возвращаем данные

    def __getitem__(self, idx):
        return (self.samples[idx], label) 
    

In [21]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        self.labels = self.data.iloc[:,-1]


    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,25*3)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
            
        

        return (item, label) 

In [22]:
dataset = Skeleton_Dataset(file_path = "skeletons_classes_1_30.csv", transform=None)

In [23]:
skel, lab = dataset.__getitem__(41)
skel, lab = dataset[41]

In [24]:
LABELS[lab]

10

In [25]:
skel.shape

(45, 75)

In [26]:
skels_for_draw = skel.reshape(-1,25,3)

Визуализация убивает ядро - закомментируем

In [27]:
# from mpl_toolkits.mplot3d import Axes3D  
# import mpl_toolkits.mplot3d as plt3d
# import matplotlib.animation as animation

# import matplotlib.pyplot as plt
# import numpy as np
# from time import sleep

# %matplotlib notebook


# bone_pairs = (
#     (1, 2), (2, 21), (3, 21), (4, 3), (5, 21), (6, 5),
#     (7, 6), (8, 7), (9, 21), (10, 9), (11, 10), (12, 11),
#     (13, 1), (14, 13), (15, 14), (16, 15), (17, 1), (18, 17),
#     (19, 18), (20, 19), (22, 23), (21, 21), (23, 8), (24, 25),(25, 12)
# )

# i = 0
# # for for_draw in skels_for_draw:
# i = i + 1
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for_draw = skels_for_draw[10]

# for pair in bone_pairs:
#     xs = for_draw[pair[0]-1][0], for_draw[pair[1]-1][0]
#     ys = for_draw[pair[0]-1][1], for_draw[pair[1]-1][1]
#     zs = for_draw[pair[0]-1][2], for_draw[pair[1]-1][2]

#     line = plt3d.art3d.Line3D(zs, xs, ys)
#     ax.add_line(line)


# for x,y,z in for_draw:
#     ax.scatter(z, x, y, color='black', marker='s')

#     ax.set_ylim((-0.3, 0.3))
#     ax.set_zlim((-1.0, 0.25))
#     ax.set_xlim((3.0, 4.0))


#     ax.set_xlabel('X Label')
#     ax.set_ylabel('Y Label')
#     ax.set_zlabel('Z Label')


#     plt.show()

# #     plt.savefig('pics_1/test_' +str(i)+'.png', bbox_inches='tight')
# #     plt.clf()

In [28]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),int(0.25*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [29]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=8, bias=True)
)

In [30]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [31]:
from torch import optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.1039  / 21 ✗ (22)
epoch : 9 iter : 41 (0m 49s) 2.0795  / 25 ✗ (27)
epoch : 19 iter : 31 (1m 32s) 2.0538  / 25 ✗ (21)
epoch : 29 iter : 21 (2m 16s) 1.7181  / 22 ✓
epoch : 39 iter : 11 (3m 1s) 1.6409  / 8 ✗ (25)
epoch : 49 iter : 1 (3m 45s) 1.7392  / 8 ✓
epoch : 58 iter : 42 (4m 26s) 1.4541  / 31 ✓
epoch : 68 iter : 32 (5m 11s) 1.2905  / 23 ✗ (31)
epoch : 78 iter : 22 (5m 57s) 1.4027  / 10 ✗ (21)
epoch : 88 iter : 12 (6m 44s) 1.0329  / 8 ✓
epoch : 98 iter : 2 (7m 28s) 1.3730  / 10 ✗ (21)
epoch : 107 iter : 43 (8m 11s) 1.3087  / 23 ✗ (8)
epoch : 117 iter : 33 (8m 57s) 1.2002  / 21 ✗ (10)
epoch : 127 iter : 23 (9m 39s) 0.9730  / 23 ✗ (31)
epoch : 137 iter : 13 (10m 21s) 0.7296  / 22 ✓
epoch : 147 iter : 3 (11m 3s) 0.7874  / 23 ✗ (31)
epoch : 156 iter : 44 (11m 45s) 0.6231  / 22 ✓
epoch : 166 iter : 34 (12m 26s) 0.6232  / 8 ✓
epoch : 176 iter : 24 (13m 7s) 0.7999  / 23 ✗ (21)
epoch : 186 iter : 14 (13m 49s) 0.6185  / 25 ✓
epoch : 196 iter : 4 (14m 45s) 0.5103  /

In [ ]:
%matplotlib notebook
plt.plot(list(range(0,len(all_losses))),all_losses)
plt.show()

<IPython.core.display.Javascript object>